In [2]:
import cv2
from deepface import DeepFace

# Define the emotion responses
emotion_responses = {
    "happy": "That's great to see! Keep smiling! 😊",
    "sad": "I'm here for you. Remember, it's okay to feel sad. 🌧️",
    "angry": "Take a deep breath. Let's find some calm together. 🧘‍♂️",
    "surprised": "Wow! That must be something new! 😲",
    "disgust": "It's okay to feel that way. Let's focus on something positive! 😊",
    "fear": "You are safe. Let's talk about what's bothering you. 🕊️",
    "neutral": "Everything seems calm. How can I help you today? 🤗"
}

# Load the face cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rgb_frame = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2RGB)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        face_roi = rgb_frame[y:y + h, x:x + w]
        
        # Analyze the detected face for emotions
        result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
        emotion = result[0]['dominant_emotion']
        
        # Get the corresponding response
        response = emotion_responses.get(emotion, "I don't know how to respond to that.")
        
        # Draw rectangle around the face and display emotion and response
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        cv2.putText(frame, response, (x, y + h + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Display the frame with emotion and response
    cv2.imshow('Real-time Emotion Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()

